## Multivariable Linear Regression

In this tutorial, we implement linear regression for multiple parameters. 

In [9]:
import numpy as np
import pandas as pd
from ipywidgets import *
import matplotlib.pyplot as plt

df = pd.read_csv('../data/house_prices_2.csv')

training_data = df.head() 
training_data

,area,rooms,price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


## Feature Scaling

Before using the training data, we should make it easy to work with. The values for area, rooms and price are spread over different ranges and as a consequence gradient descent will need to run for longer, over many iterations before finding the optimal values for parameters $\theta$. We can have the data spread out over approximately the same range by  **scaling and normalizing** the input variables.

#### Feature Scaling And Mean Normalization
Feature scaling involves dividing the input variable with the range -- difference between max and min values or the standard deviation and mean normalization involves substracting the mean from the input variable.

The formula below scales and normalizes features.

$x_i = \frac{x_i - \mu_i}{\sigma_i}$ where $\mu_i$ is the mean for the ith feature and $\sigma_i$ is the range or standard deviation for the ith feature. In this tutorial, we'll use standard deviation instead of range for $\sigma_i$

**Example**
Scale and normalize area's first entry

$\mu_i = 2104$

$\sigma_i = 568.82$

norm_area $= \frac{2104 - 2104}{568.83}$

Therefore, norm_area $=$ 0

The snippet below implements the formula in Python


In [16]:
def normalize(input_variables):
    result = None

    mean = np.mean(input_variables)
    std = np.std(input_variables)

    result = (input_variables - mean) / std
    return result

Now we can normalize our input variables, area and rooms

In [25]:

training_data['area'] = training_data['area'].apply(lambda x: normalize(x))
training_data

,area,rooms,price
0,NaN,3,399900
1,NaN,3,329900
2,NaN,3,369000
3,NaN,2,232000
4,NaN,4,539900
